In [2]:
pwd


'C:\\Users\\prasa\\OneDrive\\Desktop\\IBM Project'

In [ ]:
!%pip install tensorflow==2.7.1
!%pip install keras==2.2.4

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Training Datagen
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
# Testing Datagen
test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.ap.cloud-object-appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-be0yo09hakvd76'
object_key = 'conversation engine for deaf and dumb.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [ ]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
%%bash
ls Communication_Dataset

In [ ]:
x_train = train_datagen.flow_from_directory('dataset/training_set',target_size=(64,64),batch_size=900, class_mode='categorical', color_mode='grayscale')
x_test = test_datagen.flow_from_directory('dataset/test_set',target_size=(64,64),batch_size=900, class_mode='categorical', color_mode='grayscale')

In [ ]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

In [ ]:
# The Class Indices in Training Dataset
x_train.class_indices

In [ ]:
# Importing Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [ ]:
# Creating Model
model=Sequential()

In [ ]:
# Adding Layers
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Adding Hidden Layers
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

# Adding Output Layer
model.add(Dense(9,activation='softmax'))

In [ ]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

In [ ]:
model.save('aslpng1.h5')
# Current accuracy is 0.9994

In [ ]:
# Convert the Saved Model to a Tar Compressed Format
!tar -zcvf IBM_TrainedModel.tgz IBM_Communication_Model.h5

In [ ]:
%%bash
ls -ll

In [ ]:
!%pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://eu-gb.ml.cloud.ibm.com",
	
}

client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])